In [1]:
import csv
import os
from matplotlib import pyplot as plt
import cv2
import numpy as np
import numpy.random
import random
from collections import namedtuple
import tensorflow

SIDE_CAMERA_OFFSET = 0.3


class DataLoader:
    
    FrameRecord = namedtuple('FrameRecord', ['center_path', 'left_path', 'right_path', 'angle', 'dup_factor'])
    
    def __init__(
            self, data_dirs_and_dup_factors):
        self.data_dirs_and_dup_factors = data_dirs_and_dup_factors
        
        self.frame_records = []
        for d, dup_factor in self.data_dirs_and_dup_factors:
            with open(os.path.join(d, 'driving_log.csv')) as inf:
                for line in csv.reader(inf):
                    paths = line[:3]
                    paths = [os.path.join(d, 'IMG', p.split('/')[-1]) for p in paths]
                    angle = float(line[3])
                    self.frame_records.append(self.FrameRecord(*(paths + [angle, dup_factor])))

    @staticmethod
    def fix_img_path(path):
        fname = path.split('/')[-1]
        return os.path.join(os.getcwd(), 'data', 'IMG', fname)
    
    @staticmethod
    def all_img_paths(path):
        center = fix_img_path(path)
        left = 'left'.join(center.rsplit('center', 1))
        right = 'right'.join(center.rsplit('right', 1))
        
        return center, left, right
    
    def load_all_data(self):
        rv_x = []
        rv_y = []
        for r in self.frame_records:
            offsets = [0, SIDE_CAMERA_OFFSET, -SIDE_CAMERA_OFFSET]
            for path, offset in zip(r[:3], offsets):
                imdata = cv2.imread(path)
                angle = r.angle + offset
                
                for i in range(r.dup_factor):
                    rv_x.append(imdata)
                    rv_y.append(angle)
                    # also append reversed data
                    rv_x.append(imdata[:,::-1,:])
                    rv_y.append(-angle)
        
        return np.array(rv_x), np.array(rv_y)

In [2]:
data_loader = DataLoader([
    ('new_data/driving_straight', 1),
    ('new_data/driving_straight_reverse_direction', 1),
    ('new_data/driving_straight2', 1),
#     ('new_data/jungle_track', 1),
#     ('new_data/jungle_track_reverse_direction', 1),
])

x, y = data_loader.load_all_data()

print('loss for driving in the mean direction is {}'.format(
    np.mean((y - np.mean(y)) ** 2)
))

loss for driving in the mean direction is 0.06930196517655197


In [3]:
from keras import models, layers, optimizers, callbacks

EPOCHS = 7
TOP_CROP = 70
BOTTOM_CROP = 20

# get repeatable results
numpy.random.seed(42)
tensorflow.set_random_seed(42)


def nvidia_net():
    model = models.Sequential()
    model.add(layers.Cropping2D(cropping=((TOP_CROP, BOTTOM_CROP), (0, 0)), input_shape=(160, 320, 3)))
    model.add(layers.Lambda(lambda x: x / 255.0 - 0.5))
    model.add(layers.Conv2D(24, (5, 5), strides=(2, 2), activation='elu'))
    model.add(layers.Conv2D(36, (5, 5), strides=(2, 2), activation='elu'))
    model.add(layers.Conv2D(48, (5, 5), strides=(2, 2), activation='elu'))
    model.add(layers.Conv2D(64, (3, 3), activation='elu'))
    model.add(layers.Conv2D(64, (3, 3), activation='elu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(1164, activation='elu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(100, activation='elu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(50, activation='elu'))
    model.add(layers.Dense(10, activation='elu'))
    model.add(layers.Dense(1))
    return model

def lenet():
    model = models.Sequential()
    model.add(layers.Cropping2D(cropping=((TOP_CROP, BOTTOM_CROP), (0, 0)), input_shape=(160, 320, 3)))
    model.add(layers.Lambda(lambda x: x / 255.0 - 0.5))
    model.add(layers.Conv2D(6, (5, 5), activation='elu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(16, (5, 5), activation='elu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='elu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(84))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1))
    return model

def single_layer_net():
    model = models.Sequential()
    model.add(layers.Cropping2D(cropping=((TOP_CROP, BOTTOM_CROP), (0, 0)), input_shape=(160, 320, 3)))
    model.add(layers.Lambda(lambda x: x / 255.0 - 0.5))
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model


model = nvidia_net()

optimizer = optimizers.Adam() # lr=0.0001)

model.compile(optimizer=optimizer, loss='mse')

checkpointer = callbacks.ModelCheckpoint(filepath='model1_ckpt.h5', verbose=1, save_best_only=True)

model.fit(x, y, validation_split=0.2, epochs=EPOCHS, shuffle=True, callbacks=[checkpointer])

model.save('model1.h5')

Using TensorFlow backend.


Train on 18547 samples, validate on 4637 samples
Epoch 1/7
18547/18547 [==============================] - 22s 1ms/step - loss: 0.0337 - val_loss: 0.0130
Epoch 2/7
18547/18547 [==============================] - 22s 1ms/step - loss: 0.0156 - val_loss: 0.0138
Epoch 3/7
18547/18547 [==============================] - 22s 1ms/step - loss: 1371.6312 - val_loss: 0.1061
Epoch 4/7
18547/18547 [==============================] - 22s 1ms/step - loss: 0.0940 - val_loss: 0.0839
Epoch 5/7
18547/18547 [==============================] - 22s 1ms/step - loss: 0.0779 - val_loss: 0.0734
Epoch 6/7
18547/18547 [==============================] - 22s 1ms/step - loss: 0.0713 - val_loss: 0.0700
Epoch 7/7
18547/18547 [==============================] - 22s 1ms/step - loss: 0.0696 - val_loss: 0.0693
